<a href="https://colab.research.google.com/github/w-e-s-l-e-y/BairesDev---Machine-Learning-Practitioner/blob/main/BinarizacaoImagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def carregar_imagem_ppm(caminho):
    """
    Carrega uma imagem PPM (formato P3) e a converte em uma matriz 3D RGB.
    Lida com erros de decodificação e tenta diferentes encodings.
    """
    try:
        with open(caminho, 'rb') as f:
            linhas = f.readlines()

        # Tentativa de decodificação com encodings comuns
        encodings = ['utf-8', 'latin-1', 'cp1252']
        decoded_lines = None

        for enc in encodings:
            try:
                decoded_lines = [linha.decode(enc).strip() for linha in linhas if not linha.startswith(b'#')]
                if decoded_lines[0] == 'P3':
                    break
            except UnicodeDecodeError:
                continue

        if decoded_lines is None:
            raise ValueError("Não foi possível decodificar o arquivo PPM com nenhum dos encodings: " + ", ".join(encodings))

        # Ignorar comentários e obter cabeçalho
        assert decoded_lines[0] == 'P3', "Formato PPM inválido! Use imagens P3."

        # Dimensões da imagem e valor máximo
        largura, altura = map(int, decoded_lines[1].split())
        max_val = int(decoded_lines[2])

        # Dados de pixels
        pixels = decoded_lines[3:]
        pixels = [int(valor) for valor in " ".join(pixels).split()]

        # Converter para matriz 3D
        imagem = []
        for i in range(altura):
            linha = []
            for j in range(largura):
                idx = (i * largura + j) * 3
                linha.append([pixels[idx], pixels[idx + 1], pixels[idx + 2]])
            imagem.append(linha)

        return imagem, largura, altura, max_val

    except FileNotFoundError:
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    except Exception as e:
        raise Exception(f"Erro ao carregar imagem PPM: {e}")

def salvar_imagem_ppm(caminho, imagem, largura, altura, max_val=255, tons_de_cinza=False):
    """
    Salva uma matriz de imagem em um arquivo PPM (formato P3).
    Aceita matrizes RGB (3D) ou em tons de cinza (2D).
    """
    with open(caminho, 'w') as f:
        if tons_de_cinza:
            f.write("P2\n")  # P2 para tons de cinza
        else:
            f.write("P3\n")  # P3 para RGB
        f.write(f"{largura} {altura}\n")
        f.write(f"{max_val}\n")

        for linha in imagem:
            if tons_de_cinza:
                f.write(" ".join(map(str, linha)) + "\n")
            else:
                for pixel in linha:
                    f.write(f"{pixel[0]} {pixel[1]} {pixel[2]} ")
                f.write("\n")

def rgb_para_tons_de_cinza(imagem):
    """
    Converte uma imagem RGB para tons de cinza.
    """
    altura = len(imagem)
    largura = len(imagem[0])
    tons_de_cinza = [[0] * largura for _ in range(altura)]

    for i in range(altura):
        for j in range(largura):
            r, g, b = imagem[i][j]
            tons_de_cinza[i][j] = int(0.299 * r + 0.587 * g + 0.114 * b)

    return tons_de_cinza

def tons_de_cinza_para_binario(imagem, limiar=128):
    """
    Converte uma imagem em tons de cinza para binário (preto e branco).
    """
    altura = len(imagem)
    largura = len(imagem[0])
    binario = [[0] * largura for _ in range(altura)]

    for i in range(altura):
        for j in range(largura):
            binario[i][j] = 255 if imagem[i][j] >= limiar else 0

    return binario

if __name__ == "__main__":
    # Montar o Google Drive e acessar os arquivos
    from google.colab import drive
    drive.mount('/content/drive')

    # Defina os caminhos de entrada e saída
    caminho_entrada = "/content/drive/MyDrive/Colab Notebooks/butterfly.ppm"  # Substitua pelo caminho correto da sua imagem
    caminho_saida_cinza = "/content/drive/MyDrive/Colab Notebooks/butterfly_cinza.ppm"
    caminho_saida_binaria = "/content/drive/MyDrive/Colab Notebooks/butterfly_binaria.ppm"

    # Carregar a imagem do Google Drive
    try:
        imagem, largura, altura, max_val = carregar_imagem_ppm(caminho_entrada)
        print(f"Imagem carregada com sucesso de {caminho_entrada}")

        # Converter para tons de cinza
        imagem_cinza = rgb_para_tons_de_cinza(imagem)
        salvar_imagem_ppm(caminho_saida_cinza, imagem_cinza, largura, altura, tons_de_cinza=True)
        print(f"Imagem em tons de cinza salva em: {caminho_saida_cinza}")

        # Converter para binário (preto e branco)
        imagem_binaria = tons_de_cinza_para_binario(imagem_cinza, limiar=128)
        salvar_imagem_ppm(caminho_saida_binaria, imagem_binaria, largura, altura, tons_de_cinza=True)
        print(f"Imagem binarizada salva em: {caminho_saida_binaria}")

    except (FileNotFoundError, ValueError, Exception) as e:
        print(f"Erro: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imagem carregada com sucesso de /content/drive/MyDrive/Colab Notebooks/butterfly.ppm
Imagem em tons de cinza salva em: /content/drive/MyDrive/Colab Notebooks/butterfly_cinza.ppm
Imagem binarizada salva em: /content/drive/MyDrive/Colab Notebooks/butterfly_binaria.ppm
